In [2]:
pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 1.1 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 1.0 MB/s eta 0:00:003 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from transformers import BeitForImageClassification, BeitConfig
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm
from pathlib import Path
import time
import json

class BEiTTrainer:
    def __init__(
        self,
        data_dir,
        num_classes,
        batch_size=32,
        learning_rate=5e-5,
        train_split=0.8,
        image_size=224,
        num_workers=10,
        checkpoint_dir='checkpoints'
    ):
        self.data_dir = data_dir
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.train_split = train_split
        self.image_size = image_size
        self.num_workers = num_workers
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(exist_ok=True)
        
        self.log_dir = Path('training_logs')
        self.log_dir.mkdir(exist_ok=True)
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        self._setup_data()
        self._setup_model()
        self._setup_training()

    def _setup_data(self):
        # BEiT specific transforms
        data_transform = transforms.Compose([
            transforms.Resize((self.image_size, self.image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # BEiT normalization
        ])

        # Load dataset
        full_dataset = datasets.ImageFolder(self.data_dir, transform=data_transform)
        
        # Calculate splits
        train_size = int(self.train_split * len(full_dataset))
        val_size = len(full_dataset) - train_size
        
        # Create splits with fixed seed
        generator = torch.Generator().manual_seed(42)
        self.train_dataset, self.val_dataset = random_split(
            full_dataset, [train_size, val_size], generator=generator
        )
        
        # Calculate class weights for balanced sampling
        labels = [full_dataset.targets[i] for i in self.train_dataset.indices]
        class_counts = torch.bincount(torch.tensor(labels))
        class_weights = 1.0 / class_counts.float()
        sample_weights = [class_weights[label] for label in labels]
        self.sampler = torch.utils.data.WeightedRandomSampler(
            sample_weights, len(sample_weights), replacement=True
        )
        
        # Create data loaders with balanced sampling
        self.train_loader = DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            sampler=self.sampler,
            num_workers=self.num_workers,
            pin_memory=True
        )
        
        self.val_loader = DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
            pin_memory=True
        )

    def _setup_model(self):
        # Initialize BEiT model with custom configuration
        config = BeitConfig.from_pretrained(
            "microsoft/beit-base-patch16-224-pt22k-ft22k",
            num_labels=self.num_classes,
            id2label={str(i): str(i) for i in range(self.num_classes)},
            label2id={str(i): i for i in range(self.num_classes)}
        )
        
        self.model = BeitForImageClassification.from_pretrained(
            "microsoft/beit-base-patch16-224-pt22k-ft22k",
            config=config,
            ignore_mismatched_sizes=True
        )
        
        # Progressive unfreezing of layers
        for param in self.model.parameters():
            param.requires_grad = False
            
        # Unfreeze specific layers for fine-tuning
        trainable_layers = [
            'pooler',
            'classifier',
            'encoder.layer.11',
            'encoder.layer.10',
            'encoder.layer.9',
            'encoder.layer.8',
            'encoder.layer.7',
            'encoder.layer.6',
            'encoder.layer.5'
        ]
        
        for name, param in self.model.named_parameters():
            if any(layer in name for layer in trainable_layers):
                param.requires_grad = True
                
        self.model = self.model.to(self.device)

    def _setup_training(self):
        # Use label smoothing and class weights
        class_counts = torch.bincount(torch.tensor(self.train_dataset.dataset.targets))
        class_weights = 1.0 / class_counts.float()
        class_weights = class_weights.to(self.device)
        self.criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
        
        # Separate parameter groups with different learning rates
        encoder_params = []
        head_params = []
        
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                if any(x in name for x in ['pooler', 'classifier']):
                    head_params.append(param)
                else:
                    encoder_params.append(param)
        
        # Modified optimizer settings
        self.optimizer = optim.AdamW([
            {'params': encoder_params, 'lr': self.learning_rate * 0.1},
            {'params': head_params, 'lr': self.learning_rate}
        ], weight_decay=0.02)
        
        # Cosine annealing with warm restarts
        self.scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer,
            T_0=5,
            T_mult=2,
            eta_min=1e-6
        )
        
        self.scaler = GradScaler()

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        pbar = tqdm(self.train_loader, desc='Training')
        for batch_idx, (images, targets) in enumerate(pbar):
            images, targets = images.to(self.device), targets.to(self.device)
            
            self.optimizer.zero_grad(set_to_none=True)
            
            with autocast():
                outputs = self.model(images)
                loss = self.criterion(outputs.logits, targets)
            
            self.scaler.scale(loss).backward()
            
            # Gradient clipping
            self.scaler.unscale_(self.optimizer)
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.scaler.step(self.optimizer)
            self.scaler.update()
            
            total_loss += loss.item()
            _, predicted = outputs.logits.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_postfix({
                'loss': total_loss / (batch_idx + 1),
                'acc': 100. * correct / total
            })
        
        self.scheduler.step()
        return total_loss / len(self.train_loader), 100. * correct / total

    @torch.no_grad()
    def validate(self):
        self.model.eval()
        total_loss = 0
        correct = 0
        total = 0
        class_correct = torch.zeros(self.num_classes, device=self.device)
        class_total = torch.zeros(self.num_classes, device=self.device)
        
        for images, targets in tqdm(self.val_loader, desc='Validating'):
            images, targets = images.to(self.device), targets.to(self.device)
            
            with autocast():
                outputs = self.model(images)
                loss = self.criterion(outputs.logits, targets)
            
            total_loss += loss.item()
            _, predicted = outputs.logits.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            # Per-class accuracy
            for i in range(self.num_classes):
                mask = targets == i
                class_correct[i] += predicted[mask].eq(targets[mask]).sum().item()
                class_total[i] += mask.sum().item()
        
        # Print per-class accuracy
        for i in range(self.num_classes):
            if class_total[i] > 0:
                print(f'Accuracy of class {i}: {100 * class_correct[i] / class_total[i]:.2f}%')
        
        return total_loss / len(self.val_loader), 100. * correct / total

    def train(self, num_epochs=50):
        print("Starting training...")
        start_time = time.time()
        best_val_acc = 0
        patience = 10
        no_improve = 0
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch + 1}/{num_epochs}")
            
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()
            
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                self.save_checkpoint(epoch, val_loss, val_acc, is_best=True)
                no_improve = 0
                print("New best model saved!")
            else:
                no_improve += 1
                if no_improve >= patience:
                    print("Early stopping triggered!")
                    break
            
            if (epoch + 1) % 5 == 0:
                self.save_checkpoint(epoch, val_loss, val_acc)
        
        print(f"\nTraining completed. Best validation accuracy: {best_val_acc:.2f}%")

    def save_checkpoint(self, epoch, val_loss, val_acc, is_best=False):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc
        }
        
        if is_best:
            torch.save(checkpoint, self.checkpoint_dir / 'best_model.pth')
        else:
            torch.save(checkpoint, self.checkpoint_dir / f'checkpoint_epoch_{epoch}.pth')

# Usage
if __name__ == "__main__":
    trainer = BEiTTrainer(
        data_dir="/home/storage/Backup_E/phd/Object_detection/Data/final_augmented (Copy)",
        num_classes=25,
        batch_size=32,
        learning_rate=5e-5
    )
    trainer.train(num_epochs=50)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2025-01-11 10:23:09.052539: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-11 10:23:09.187759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-11 10:23:09.241344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-11 10:23:09.257444: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-11 10:23:09.356266: I tensorflow/core/platform/cpu_feature_guar

Using device: cuda


config.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

Accuracy of class 0: 96.91%
Accuracy of class 1: 96.80%
Accuracy of class 2: 84.98%
Accuracy of class 3: 83.87%
Accuracy of class 4: 83.21%
Accuracy of class 5: 93.28%
Accuracy of class 6: 98.30%
Accuracy of class 7: 95.83%
Accuracy of class 8: 83.81%
Accuracy of class 9: 72.78%
Accuracy of class 10: 93.82%
Accuracy of class 11: 83.17%
Accuracy of class 12: 78.88%
Accuracy of class 13: 64.28%
Accuracy of class 14: 87.44%
Accuracy of class 15: 85.13%
Accuracy of class 16: 92.68%
Accuracy of class 17: 97.11%
Accuracy of class 18: 97.45%
Accuracy of class 19: 99.24%
Accuracy of class 20: 96.94%
Accuracy of class 21: 97.12%
Accuracy of class 22: 97.43%
Accuracy of class 23: 99.30%
Accuracy of class 24: 98.79%
Train Loss: 0.6073, Train Acc: 98.73%
Val Loss: 1.1245, Val Acc: 89.39%

Epoch 49/50


Validating: 100%|███████████████████████████| 1216/1216 [01:36<00:00, 12.65it/s]


Accuracy of class 0: 97.24%
Accuracy of class 1: 96.22%
Accuracy of class 2: 81.80%
Accuracy of class 3: 88.89%
Accuracy of class 4: 81.59%
Accuracy of class 5: 93.78%
Accuracy of class 6: 98.17%
Accuracy of class 7: 96.06%
Accuracy of class 8: 84.44%
Accuracy of class 9: 73.28%
Accuracy of class 10: 95.42%
Accuracy of class 11: 83.78%
Accuracy of class 12: 79.05%
Accuracy of class 13: 67.36%
Accuracy of class 14: 88.05%
Accuracy of class 15: 87.05%
Accuracy of class 16: 90.22%
Accuracy of class 17: 94.77%
Accuracy of class 18: 98.54%
Accuracy of class 19: 99.13%
Accuracy of class 20: 97.05%
Accuracy of class 21: 97.68%
Accuracy of class 22: 96.71%
Accuracy of class 23: 98.60%
Accuracy of class 24: 98.79%
Train Loss: 0.6067, Train Acc: 98.80%
Val Loss: 1.1212, Val Acc: 89.65%
New best model saved!

Epoch 50/50


Validating: 100%|███████████████████████████| 1216/1216 [01:28<00:00, 13.70it/s]


Accuracy of class 0: 96.10%
Accuracy of class 1: 97.71%
Accuracy of class 2: 83.02%
Accuracy of class 3: 84.40%
Accuracy of class 4: 86.58%
Accuracy of class 5: 94.68%
Accuracy of class 6: 98.30%
Accuracy of class 7: 95.66%
Accuracy of class 8: 83.94%
Accuracy of class 9: 67.12%
Accuracy of class 10: 94.90%
Accuracy of class 11: 81.26%
Accuracy of class 12: 81.37%
Accuracy of class 13: 66.98%
Accuracy of class 14: 85.38%
Accuracy of class 15: 90.01%
Accuracy of class 16: 88.76%
Accuracy of class 17: 94.97%
Accuracy of class 18: 98.30%
Accuracy of class 19: 99.13%
Accuracy of class 20: 97.85%
Accuracy of class 21: 98.18%
Accuracy of class 22: 97.51%
Accuracy of class 23: 99.30%
Accuracy of class 24: 98.79%
Train Loss: 0.6047, Train Acc: 98.83%
Val Loss: 1.1292, Val Acc: 89.41%

Training completed. Best validation accuracy: 89.65%


In [7]:
# Create directory if it doesn't exist
import os
os.makedirs('saved_models', exist_ok=True)

# Save the complete model (not just state_dict)
torch.save(trainer.model, 'saved_models/best_model_biet.pt')

In [8]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import re

class TrainingMetricsPlotter:
    def __init__(self, checkpoint_dir):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.train_losses = []
        self.train_accuracies = []
        self.val_losses = []
        self.val_accuracies = []
        self.epochs = []
        
    def natural_sort_key(self, checkpoint_path):
        """Extract epoch number for proper sorting"""
        match = re.search(r'epoch_(\d+)', str(checkpoint_path))
        if match:
            return int(match.group(1))
        return 0
        
    def load_checkpoints(self):
        """Load metrics from all checkpoint files"""
        # Get all checkpoint files and sort them properly
        checkpoint_files = list(self.checkpoint_dir.glob("checkpoint_epoch_*.pth"))
        checkpoint_files.sort(key=self.natural_sort_key)
        
        print(f"Found {len(checkpoint_files)} checkpoint files")
        
        # Store temporary data
        temp_data = []
        
        for checkpoint_file in checkpoint_files:
            checkpoint = torch.load(checkpoint_file, map_location='cpu')
            epoch = checkpoint['epoch']
            
            # Store data in temporary list
            temp_data.append({
                'epoch': epoch,
                'val_loss': checkpoint['val_loss'],
                'val_acc': checkpoint['val_acc']
            })
        
        # Sort by epoch number to ensure correct order
        temp_data.sort(key=lambda x: x['epoch'])
        
        # Extract sorted data
        self.epochs = [data['epoch'] for data in temp_data]
        self.val_losses = [data['val_loss'] for data in temp_data]
        self.val_accuracies = [data['val_acc'] for data in temp_data]
        
        print(f"Processed {len(self.epochs)} epochs")
            
    def plot_metrics(self, save_dir='plots'):
        """Plot and save training metrics"""
        if not self.epochs:
            print("No data loaded. Please run load_checkpoints() first.")
            return
            
        save_dir = Path(save_dir)
        save_dir.mkdir(exist_ok=True)
        
        # Create figure with two subplots
        plt.style.use('ggplot')  # Using ggplot style for better visualization
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 14))
        
        # Plot losses
        # Apply moving average to smooth the curves
        window_size = 3
        val_losses_smooth = np.convolve(self.val_losses, 
                                      np.ones(window_size)/window_size, 
                                      mode='valid')
        epochs_smooth = self.epochs[window_size-1:]
        
        # Original data with light color
        ax1.plot(self.epochs, self.val_losses, 'r-', alpha=0.3, label='Original Val Loss')
        # Smoothed data with darker color
        ax1.plot(epochs_smooth, val_losses_smooth, 'r-', linewidth=2, 
                label='Smoothed Val Loss')
        
        ax1.set_title('Validation Loss Over Time', fontsize=14, pad=20)
        ax1.set_xlabel('Epoch', fontsize=12)
        ax1.set_ylabel('Loss', fontsize=12)
        ax1.legend(fontsize=10)
        ax1.grid(True, linestyle='--', alpha=0.7)
        ax1.tick_params(axis='both', labelsize=10)
        
        # Plot accuracies
        # Apply moving average to smooth the curves
        val_acc_smooth = np.convolve(self.val_accuracies, 
                                   np.ones(window_size)/window_size, 
                                   mode='valid')
        
        # Original data with light color
        ax2.plot(self.epochs, self.val_accuracies, 'b-', alpha=0.3, 
                label='Original Val Accuracy')
        # Smoothed data with darker color
        ax2.plot(epochs_smooth, val_acc_smooth, 'b-', linewidth=2, 
                label='Smoothed Val Accuracy')
        
        ax2.set_title('Validation Accuracy Over Time', fontsize=14, pad=20)
        ax2.set_xlabel('Epoch', fontsize=12)
        ax2.set_ylabel('Accuracy (%)', fontsize=12)
        ax2.legend(fontsize=10)
        ax2.grid(True, linestyle='--', alpha=0.7)
        ax2.tick_params(axis='both', labelsize=10)
        
        # Adjust layout and save
        plt.tight_layout()
        
        # Save plot
        plot_path = save_dir / 'training_metrics.png'
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"\nPlot saved to: {plot_path}")
        plt.close()
        
        # Print best metrics
        best_val_acc = max(self.val_accuracies)
        best_epoch = self.epochs[np.argmax(self.val_accuracies)]
        print(f"\nBest validation accuracy: {best_val_acc:.2f}% at epoch {best_epoch}")

        # Save metrics to text file
        metrics_path = save_dir / 'training_metrics.txt'
        with open(metrics_path, 'w') as f:
            f.write("Epoch\tVal Loss\tVal Acc\n")
            for i in range(len(self.epochs)):
                f.write(f"{self.epochs[i]}\t{self.val_losses[i]:.4f}\t"
                       f"{self.val_accuracies[i]:.2f}\n")
        print(f"Metrics saved to: {metrics_path}")

# Usage example
if __name__ == "__main__":
    plotter = TrainingMetricsPlotter('checkpoints')
    plotter.load_checkpoints()
    plotter.plot_metrics()

Found 56 checkpoint files


/tmp/ipykernel_6190/1794840932.py:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_file, map_location='cpu')


Processed 56 epochs

Plot saved to: plots/training_metrics.png

Best validation accuracy: 89.41% at epoch 49
Metrics saved to: plots/training_metrics.txt
